In [10]:
import pandas as pd
df=pd.read_csv('../database/inventario_reportes.csv', delimiter=",")
import re
def clean_text(text):
    text=str(text).lower()
    text=re.sub(r'[^\w\s]', '', text)
    return text.strip()

df['Nombre_limpio']=df['CLAVE_REP'].apply(clean_text)
df['Descripcion_limpia']=df['DESCRIPCION_ESP'].apply(clean_text)
df['clave']=df['CLAVE_REP'].apply(clean_text)

In [16]:
from sentence_transformers import SentenceTransformer

model=SentenceTransformer("sentence-transformers/LaBSE")


df['texto_completo']=df['Nombre_limpio']
embeddings=model.encode(df['texto_completo'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [17]:
import faiss
import numpy as np

# Construcción del índice (unchanged)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype(np.float32))

def buscar_registros(consulta, top_k=50):
    # Limpieza y obtención del embedding de la consulta
    consulta_limpia = clean_text(consulta)
    embedding_consulta = model.encode([consulta_limpia]).astype(np.float32)

    # Búsqueda de los k vecinos más cercanos
    distancias, indices = index.search(embedding_consulta, top_k)

    # Conversión de distancia L2 a similitud (en rango aproximado 0–1)
    similitudes = 1 - distancias[0] / 4

    # Construcción del DataFrame de resultados
    resultados = df.iloc[indices[0]].copy()
    resultados['similitud'] = similitudes

    # Cálculo del umbral: el promedio de todas las similitudes obtenidas
    umbral = similitudes.mean()

    # Filtrar solo aquellos con similitud > promedio
    resultados_filtrados = resultados[resultados['similitud'] > umbral]

    # Ordenar de mayor a menor similitud
    return resultados_filtrados.sort_values(by='similitud', ascending=False)

In [18]:
from tabulate import tabulate

resultados = buscar_registros("Que reportes son RC0")

# Selecciona solo las columnas necesarias
tabla = resultados[['CLAVE_REP', 'CLAVE_REG', 'DESCRIPCION_ESP', 'similitud']]

# Imprime la tabla con encabezados y formato elegante
with open("../test/table.txt", "w") as file:
    file.write(tabulate(tabla, headers='keys', tablefmt='fancy_grid', showindex=False))